In [41]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
def load_data(filename="daily_log.csv"):
    df = pd.read_csv(filename, parse_dates=["Date"])
    df = df.sort_values("Date")
    return df

df = load_data()
# ---------- Recent Table (last 7 days) ----------
# Transposed display: metrics as rows, dates as columns
#print("Transposed Table (Metrics ↓ vs Dates →):")
transposed_df = df.set_index("Date").T
display(transposed_df)


Date,2025-05-30,2025-05-31
Focus,2.0,3.0
Anxiety,4.0,4.0
Stress,4.0,4.0
Sleep Quality,4.0,4.0
Fatigue,4.0,3.0
Todo list,4.0,5.0
Overnight HRV,3.0,4.0
Resting HR (bpm),56.0,54.0
Training Load,135.0,NaN
Calories in (kcal),2759.0,NaN


In [42]:
import pandas as pd
import os
import csv
from datetime import date
import ipywidgets as widgets
from IPython.display import display, clear_output

# File path for data
filename = "daily_log.csv"

# --------- Subjective Metrics and their anchors ---------
metrics = {
    "Focus": ['1: Scattered', '2: Distracted', '3: Average', '4: Focused', '5: Laser-focused'],
    "Anxiety": ['1: Overwhelmed', '2: High', '3: Notable', '4: Slight', '5: Calm'],
    "Stress": ['1: Burned out', '2: High', '3: Moderate', '4: Mild', '5: Relaxed'],
    "Sleep Quality": ['1: Poor', '2: Light', '3: Okay', '4: Good', '5: Deep & Rested'],
    "Fatigue": ['1: Exhausted', '2: Heavy', '3: Moderate', '4: Light', '5: Energized'],
    "Todo list": ['1: 0 Day', '2: Missed 3', '3: Missed 2', '4: Missed 1', '5: Completed'],
    "Overnight HRV": ['1: Low', '2: Reduced', '3: Normal-low', '4: Normal-high', '5: Elevated'],
}

# --------- Objective Metrics ---------
objective_metrics = [
    "Resting HR (bpm)",
    "Training Load",
    "Calories in (kcal)",
    "Adjusted CI goal (kcal)",
    "Calories out (kcal)",
    "Net calories (kcal)",
    "Carbs (g)",
    "Fat (g)",
    "Protein (g)",
    "Cholesterol (mg)",
    "Sodium (mg)",
    "Sugar (g)",
    "Fiber (g)",
    "Hydration (L)",
    "Weight (kg)",
    "Time to bed (h:m)",
    "Time fell asleep (h:m)",
    "Time woke up (h:m)",
    "Additional notes"
]

# --------- Helper for anchors ---------
def anchor_line(labels):
    return ' | '.join(labels)

# --------- Widgets ---------
date_input = widgets.DatePicker(value=date.today(), description="Date:")
subjective_widgets = {}
objective_widgets = {}

# Subjective sliders + anchors
for m, anchors in metrics.items():
    slider = widgets.IntSlider(min=1, max=5, description=m, layout=widgets.Layout(width='300px'))
    anchors_label = widgets.HTML(f"<small style='color:#666;font-family: monospace;'>{anchor_line(anchors)}</small>")
    subjective_widgets[m] = widgets.HBox([slider, anchors_label])

# Objective inputs (text)
for m in objective_metrics:
    objective_widgets[m] = widgets.Text(
        description=m,
        layout=widgets.Layout(width='500px'),
        style={'description_width': '200px'}
    )

submit_button = widgets.Button(description="Log Today's Data", button_style='success')
output = widgets.Output()

header = ["Date"] + list(metrics.keys()) + objective_metrics

# Create CSV if needed
if not os.path.exists(filename):
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)

# Load existing data
def load_existing_data():
    if os.path.exists(filename):
        return pd.read_csv(filename, parse_dates=["Date"])
    else:
        return pd.DataFrame(columns=header)

df = load_existing_data()

# --------- Populate Fields if Date Exists ---------
def populate_fields(change):
    selected_date = pd.to_datetime(change['new'])
    row = df[df['Date'] == selected_date]

    if not row.empty:
        row = row.iloc[0]
        for m in metrics.keys():
            val = row[m]
            if pd.notna(val):
                subjective_widgets[m].children[0].value = int(val)

        for m in objective_metrics:
            val = row[m]
            if pd.notna(val):
                objective_widgets[m].value = str(val)
            else:
                objective_widgets[m].value = ""
    else:
        # Clear all fields if no entry exists
        for m in metrics.keys():
            subjective_widgets[m].children[0].value = 3  # default to middle
        for m in objective_metrics:
            objective_widgets[m].value = ""

# Connect change event
date_input.observe(populate_fields, names='value')
populate_fields({'new': date_input.value})  # populate initially

# --------- On Submit ---------
def on_submit(b):
    global df
    with output:
        clear_output()
        new_data = {"Date": pd.to_datetime(date_input.value)}

        for m in metrics.keys():
            new_data[m] = subjective_widgets[m].children[0].value
        for m in objective_metrics:
            val = objective_widgets[m].value
            try:
                new_data[m] = float(val) if val != '' else None
            except:
                new_data[m] = val

        # Update or append
        if new_data["Date"] in df["Date"].values:
            df.loc[df["Date"] == new_data["Date"], new_data.keys()] = pd.Series(new_data)
            print(f"Updated entry for {new_data['Date'].date()}")
        else:
            df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
            print(f"Added new entry for {new_data['Date'].date()}")

        df.to_csv(filename, index=False)
        print("Data saved successfully ✅")

submit_button.on_click(on_submit)

# --------- Layout ---------
form_items = [date_input] + list(subjective_widgets.values()) + list(objective_widgets.values()) + [submit_button, output]
form = widgets.VBox(form_items)

display(form)
